# Create a SQLite DataBase 
This database creation requires the installation of Spatialite  
https://docs.djangoproject.com/en/5.0/ref/contrib/gis/install/spatialite/


In [45]:
import sqlite3
import pandas as pd
import time

# Paths to the files
counties_csv_path = 'Resources/california_counties_forDB.csv'
extracted_data_csv_path = 'Outputs/extracted_data.csv'  # Updated to use the uploaded file
fires_cleaned_csv_path = 'Outputs/fires_2008_2022_cleaned.csv'

# Function to execute SQL with retries
def execute_with_retry(conn, sql, retries=5, delay=1):
    for attempt in range(retries):
        try:
            conn.execute(sql)
            return
        except sqlite3.OperationalError as e:
            if 'database is locked' in str(e):
                time.sleep(delay)
            else:
                raise
    raise sqlite3.OperationalError("Database is still locked after retries")

# Create a SQLite database connection
conn = sqlite3.connect('TFFlaskV2/fire_data.sqlite')

# Load the california_counties dataframe
counties_df = pd.read_csv(counties_csv_path)

# Explicitly create the california_counties table
create_counties_table_sql = """
CREATE TABLE IF NOT EXISTS california_counties (
    County TEXT PRIMARY KEY,
    Abbreviation TEXT
);
"""

# Execute the creation of the california_counties table
execute_with_retry(conn, create_counties_table_sql)

# Insert the data into the california_counties table
counties_df.to_sql('california_counties', conn, if_exists='replace', index=False)

# Load the extracted_data and fires_cleaned dataframes
extracted_data_df = pd.read_csv(extracted_data_csv_path)
fires_cleaned_df = pd.read_csv(fires_cleaned_csv_path)

# Check if the extracted_data and fires_cleaned tables exist, create them if they don't
def check_and_create_table(conn, table_name, df):
    query = f"SELECT name FROM sqlite_master WHERE type='table' AND name='{table_name}';"
    result = conn.execute(query).fetchone()
    if not result:
        df.to_sql(table_name, conn, if_exists='replace', index=False)

# Ensure tables exist before proceeding
check_and_create_table(conn, 'extracted_data', extracted_data_df)
check_and_create_table(conn, 'fires_cleaned', fires_cleaned_df)

# SQL statements to be executed separately
sql_statements = [
    "PRAGMA foreign_keys=off;",
    
    # Create a new extracted_data table with a foreign key constraint
    """CREATE TABLE IF NOT EXISTS extracted_data_new (
        County TEXT,
        Tot_Damage REAL,
        Year INTEGER,
        Abbreviation TEXT,
        FOREIGN KEY (County) REFERENCES california_counties (County)
    );""",
    
    "INSERT INTO extracted_data_new (County, Tot_Damage, Year, Abbreviation) SELECT County, Tot_Damage, Year, County AS Abbreviation FROM extracted_data;",
    
    "DROP TABLE IF EXISTS extracted_data;",
    
    "ALTER TABLE extracted_data_new RENAME TO extracted_data;",
    
    # Create a new fires_cleaned table with a foreign key constraint
    """CREATE TABLE IF NOT EXISTS fires_cleaned_new (
        County TEXT,
        `Fire Name` TEXT,  -- Using backticks for the column with space
        Start DATE,
        Contained DATE,
        Acres REAL,
        Deaths_FF INTEGER,
        Deaths_Civil INTEGER,
        Duration INTEGER,
        Strux_Destr INTEGER,
        Strux_Dmgd INTEGER,
        FOREIGN KEY (County) REFERENCES california_counties (County)
    );""",
    
    "INSERT INTO fires_cleaned_new (County, `Fire Name`, Start, Contained, Acres, Deaths_FF, Deaths_Civil, Duration, Strux_Destr, Strux_Dmgd) "
    "SELECT County, `Fire Name`, Start, Contained, Acres, Deaths_FF, Deaths_Civil, Duration, Strux_Destr, Strux_Dmgd FROM fires_cleaned;",
    
    "DROP TABLE IF EXISTS fires_cleaned;",
    
    "ALTER TABLE fires_cleaned_new RENAME TO fires_cleaned;",
    
    "PRAGMA foreign_keys=on;"
]

# Start transaction
conn.execute("BEGIN TRANSACTION;")

# Execute each SQL statement with retry logic
for statement in sql_statements:
    execute_with_retry(conn, statement)

# Commit the changes and close the connection
conn.commit()
conn.close()

print("Database setup complete.")


Database setup complete.


In [47]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('TFFlaskV2/fire_data.sqlite')

# Update county names in extracted_data to uppercase
conn.execute("UPDATE extracted_data SET County = UPPER(County);")
conn.commit()

# Close the connection
conn.close()


In [ ]:
# Display the columns
print(extracted_data_df.columns)

print(fires_cleaned_df.columns)

print(counties_df.columns)

In [30]:
# Connect to the SQLite database
conn = sqlite3.connect('TFFlaskV2/fire_data.db')

# SQL query to retrieve fire names and counties
query = """
SELECT County, `Fire Name`
FROM fires_cleaned;
"""

# Execute the query and load the results into a pandas DataFrame
fire_names_df = pd.read_sql_query(query, conn)

# Close the connection
conn.close()

# Display the results
print(fire_names_df)

              County              Fire Name
0               LAKE  CONTROL BURN, GEYSERS
1     SAN BERNARDINO                  Bluff
2           MARIPOSA              WAWONA NW
3        LOS ANGELES            SANTA ANITA
4          RIVERSIDE                 APACHE
...              ...                    ...
1093        SISKIYOU               MOUNTAIN
1094       RIVERSIDE               FAIRVIEW
1095          PLACER               MOSQUITO
1096          MADERA                   FORK
1097           MODOC                 BARNES

[1098 rows x 2 columns]


In [44]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect('TFFlaskV2/fire_data.db')

# SQL query to retrieve total damages and fire names for 2014 in San Bernardino County
query = """
SELECT ed.Tot_Damage, fc.`Fire Name`, fc.Start, fc.Contained, fc.Acres
FROM extracted_data ed
JOIN fires_cleaned fc ON ed.County = fc.County
WHERE ed.County = 'SAN BERNARDINO'
AND ed.Year = 2014
AND strftime('%Y', fc.Start) = '2014';
"""

# Execute the query and load the results into a pandas DataFrame
san_bernardino_2014_df = pd.read_sql_query(query, conn)

# Close the connection
conn.close()

# Display the results
print(san_bernardino_2014_df)


   Tot_Damage Fire Name       Start   Contained  Acres
0      5485.0  ETIWANDA  2014-04-30  2014-05-01  2,200
